In [1]:
from eval_metrics import evaluate_representativeness,evaluate_simplicity, evaluate_diversity,evaluate_incentive_compatibility,evaluate_resistance_to_malicious_behavior,evaluate_resistance_to_collusion,evaluate_robustness

from voting_model import Simulation
from voting_model import Voter

## Voter Behaviours

In [ ]:
def run_simulation_with_voter_strategies():
    simulation = Simulation()
    simulation.initialize_round(30_000_000)
    
    # Example of setting up voters with different strategies
    simulation.round.add_voters([
        Voter(voter_id=i, op_available=1000, laziness=0.5, expertise=0.7, strategy='random') for i in range(50)
    ])
    simulation.round.add_voters([
        Voter(voter_id=i, op_available=1000, laziness=0.5, expertise=0.7, strategy='equal_distribution') for i in range(50, 100)
    ])
    simulation.round.add_voters([
        Voter(voter_id=i, op_available=1000, laziness=0.5, expertise=0.7, strategy='top_k') for i in range(100, 150)
    ])
    
    simulation.randomize_projects(600, coi_factor=0)
    
    simulation.simulate_voting()
    
    # Evaluate the results
    results = simulation.allocate_votes('median', quorum=17, min_amount=1500)
    print(results)

run_simulation_with_voter_strategies()


In [ ]:
num_voters=150
num_projects=600
max_allocation=30000000
willingness_to_spend=1
laziness_factor=0.6
expertise_factor=0.7
quorum=17
min_amount=1500


simulation = Simulation()

simulation.initialize_round(max_allocation)
simulation.randomize_voters(num_voters, willingness_to_spend, laziness_factor, expertise_factor)
simulation.randomize_projects(num_projects, coi_factor=0)
simulation.simulate_voting()    
results = simulation.allocate_votes('median', quorum, min_amount)
print(results)
data = simulation.get_project_data()

In [3]:
import pandas as pd
df=pd.DataFrame(data)
df.head(5)

,project_id,owner_id,rating,num_votes,score,token_amount
0,0,None,0.986013,0,0.000000,0.00
1,1,None,4.466459,80,340783.212468,302362.09
2,2,None,3.626825,54,93162.175855,82658.74
3,3,None,3.428485,52,50907.358320,45167.88
4,4,None,3.842013,60,57123.763511,50683.42


## Goal 1: Malicious behaviour Resistance

In [4]:
import numpy as np
from voting_model import Voter

def evaluate_resistance_to_malicious_behavior(simulation, method, quorum, min_amount, num_tests=100):
    results = []
    for _ in range(num_tests):
        baseline_allocations = run_baseline_simulation(simulation, method, quorum, min_amount)
        malicious_allocations = introduce_malicious_voter(simulation, method, quorum, min_amount)
        difference = measure_impact(baseline_allocations, malicious_allocations)
        results.append(difference)
    
    avg_difference = np.mean(results)
    return avg_difference

# Helper functions
def run_baseline_simulation(simulation, method, quorum, min_amount):
    simulation.simulate_voting()
    baseline_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    simulation.reset_round()
    return baseline_allocations

def introduce_malicious_voter(simulation, method, quorum, min_amount):
    malicious_voter = Voter(voter_id=-1, op_available=simulation.round.max_funding, laziness=0, expertise=1)
    malicious_voter.cast_vote(simulation.round.projects[0], malicious_voter.total_op)
    simulation.round.voters.append(malicious_voter)
    malicious_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    simulation.reset_round()
    return malicious_allocations

def measure_impact(baseline_allocations, malicious_allocations):
    difference = np.abs(np.array(baseline_allocations) - np.array(malicious_allocations)).sum()
    return difference

# Example usage
methods = ['mean', 'median', 'quadratic']
simulation = Simulation()
simulation.initialize_round(30_000_000)
simulation.randomize_voters(150, willingness_to_spend=1, laziness_factor=0.6, expertise_factor=0.7)
simulation.randomize_projects(600, coi_factor=0)

for method in methods:
    malicious_behavior_score = evaluate_resistance_to_malicious_behavior(simulation, method, quorum=17, min_amount=1500)
    print(f'{method} - Resistance to Malicious Behavior Score: {malicious_behavior_score}')


mean - Resistance to Malicious Behavior Score: 29999999.994000003
median - Resistance to Malicious Behavior Score: 30000000.005499996
quadratic - Resistance to Malicious Behavior Score: 29999999.9956


### Bribery

In [5]:
import numpy as np
from voting_model import Voter

def evaluate_impact_of_bribery(simulation, method, quorum, min_amount, bribe_amount, target_project_id, num_tests=100):
    results = []
    for _ in range(num_tests):
        baseline_allocations = run_baseline_simulation(simulation, method, quorum, min_amount)
        bribed_allocations = introduce_bribery(simulation, method, quorum, min_amount, bribe_amount, target_project_id)
        difference = measure_impact(baseline_allocations, bribed_allocations)
        results.append(difference)
    
    avg_difference = np.mean(results)
    return avg_difference

# Helper functions
def run_baseline_simulation(simulation, method, quorum, min_amount):
    simulation.simulate_voting()
    baseline_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    simulation.reset_round()
    return baseline_allocations

def introduce_bribery(simulation, method, quorum, min_amount, bribe_amount, target_project_id):
    # Select a voter to bribe
    bribed_voter = np.random.choice(simulation.round.voters)
    
    # Ensure the voter has enough balance to cast the bribe amount
    if bribed_voter.balance_op >= bribe_amount:
        # Reset the voter's votes and balance
        bribed_voter.reset_voter()
        
        # Cast a bribe vote for the target project
        target_project = next(p for p in simulation.round.projects if p.project_id == target_project_id)
        bribed_voter.cast_vote(target_project, bribe_amount)
        
        # Redistribute the remaining balance to other projects
        remaining_balance = bribed_voter.balance_op
        for project in simulation.round.projects:
            if project.project_id != target_project_id:
                amount = np.random.uniform(0, remaining_balance)
                bribed_voter.cast_vote(project, amount)
                remaining_balance -= amount
    
    bribed_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    simulation.reset_round()
    return bribed_allocations

def measure_impact(baseline_allocations, bribed_allocations):
    difference = np.abs(np.array(baseline_allocations) - np.array(bribed_allocations)).sum()
    return difference

# Example usage
methods = ['mean', 'median', 'quadratic']
simulation = Simulation()
simulation.initialize_round(30_000_000)
simulation.randomize_voters(150, willingness_to_spend=1, laziness_factor=0.6, expertise_factor=0.7)
simulation.randomize_projects(600, coi_factor=0)

for method in methods:
    target_project_id = np.random.choice([project.project_id for project in simulation.round.projects])
    bribery_impact_score = evaluate_impact_of_bribery(simulation, method, quorum=17, min_amount=1500, bribe_amount=10000, target_project_id=target_project_id)
    print(f'{method} - Impact of Bribery Score: {bribery_impact_score}')


mean - Impact of Bribery Score: 29999999.9923
median - Impact of Bribery Score: 30000000.0015
quadratic - Impact of Bribery Score: 29999999.9915


In [6]:
import numpy as np
from voting_model import Voter

def evaluate_impact_of_bribery(simulation, method, quorum, min_amount, bribe_amount, target_project_id, num_tests=100):
    results_voter_perspective = []
    results_project_perspective = []
    for _ in range(num_tests):
        baseline_allocations = run_baseline_simulation(simulation, method, quorum, min_amount)
        
        bribed_allocations_voter = introduce_bribery_voter_perspective(simulation, method, quorum, min_amount, bribe_amount, target_project_id)
        difference_voter = measure_impact(baseline_allocations, bribed_allocations_voter)
        results_voter_perspective.append(difference_voter)
        
        bribed_allocations_project = introduce_bribery_project_perspective(simulation, method, quorum, min_amount, bribe_amount, target_project_id)
        difference_project = measure_impact(baseline_allocations, bribed_allocations_project)
        results_project_perspective.append(difference_project)
    
    avg_difference_voter = np.mean(results_voter_perspective)
    avg_difference_project = np.mean(results_project_perspective)
    
    return avg_difference_voter, avg_difference_project

# Helper functions
def run_baseline_simulation(simulation, method, quorum, min_amount):
    simulation.simulate_voting()
    baseline_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    simulation.reset_round()
    return baseline_allocations

def introduce_bribery_voter_perspective(simulation, method, quorum, min_amount, bribe_amount, target_project_id):
    bribed_voter = np.random.choice(simulation.round.voters)
    
    if bribed_voter.balance_op >= bribe_amount:
        bribed_voter.reset_voter()
        target_project = next(p for p in simulation.round.projects if p.project_id == target_project_id)
        bribed_voter.cast_vote(target_project, bribe_amount)
        
        remaining_balance = bribed_voter.balance_op
        for project in simulation.round.projects:
            if project.project_id != target_project_id:
                amount = np.random.uniform(0, remaining_balance)
                bribed_voter.cast_vote(project, amount)
                remaining_balance -= amount
    
    bribed_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    simulation.reset_round()
    return bribed_allocations

def introduce_bribery_project_perspective(simulation, method, quorum, min_amount, bribe_amount, target_project_id):
    project_owner_voter = np.random.choice(simulation.round.voters)
    
    if project_owner_voter.balance_op >= bribe_amount:
        project_owner_voter.reset_voter()
        target_project = next(p for p in simulation.round.projects if p.project_id == target_project_id)
        project_owner_voter.cast_vote(target_project, bribe_amount)
        
        remaining_balance = project_owner_voter.balance_op
        for project in simulation.round.projects:
            if project.project_id != target_project_id:
                amount = np.random.uniform(0, remaining_balance)
                project_owner_voter.cast_vote(project, amount)
                remaining_balance -= amount
    
    bribed_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    simulation.reset_round()
    return bribed_allocations

def measure_impact(baseline_allocations, bribed_allocations):
    difference = np.abs(np.array(baseline_allocations) - np.array(bribed_allocations)).sum()
    return difference

# Example usage
methods = ['mean', 'median', 'quadratic']
simulation = Simulation()
simulation.initialize_round(30_000_000)
simulation.randomize_voters(150, willingness_to_spend=1, laziness_factor=0.6, expertise_factor=0.7)
simulation.randomize_projects(600, coi_factor=0)

for method in methods:
    target_project_id = np.random.choice([project.project_id for project in simulation.round.projects])
    avg_difference_voter, avg_difference_project = evaluate_impact_of_bribery(
        simulation, method, quorum=17, min_amount=1500, bribe_amount=10000, target_project_id=target_project_id
    )
    print(f'{method} - Impact of Bribery (Voter Perspective): {avg_difference_voter}')
    print(f'{method} - Impact of Bribery (Project Perspective): {avg_difference_project}')


mean - Impact of Bribery (Voter Perspective): 29999999.997599993
mean - Impact of Bribery (Project Perspective): 29999999.997599993
median - Impact of Bribery (Voter Perspective): 29999999.9929
median - Impact of Bribery (Project Perspective): 29999999.9929
quadratic - Impact of Bribery (Voter Perspective): 30000000.0072
quadratic - Impact of Bribery (Project Perspective): 30000000.0072


## Resistance to collusion

In [ ]:


def evaluate_impact_of_collusion(simulation, method, quorum, min_amount, collusion_group_size, target_project_ids, num_tests=100):
    results = []
    for _ in range(num_tests):
        baseline_allocations = run_baseline_simulation(simulation, method, quorum, min_amount)
        colluded_allocations = introduce_collusion(simulation, method, quorum, min_amount, collusion_group_size, target_project_ids)
        difference = measure_impact(baseline_allocations, colluded_allocations)
        results.append(difference)
    
    avg_difference = np.mean(results)
    return avg_difference

# Helper functions
def run_baseline_simulation(simulation, method, quorum, min_amount):
    simulation.simulate_voting()
    baseline_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    simulation.reset_round()
    return baseline_allocations

def introduce_collusion(simulation, method, quorum, min_amount, collusion_group_size, target_project_ids):
    colluding_voters = np.random.choice(simulation.round.voters, size=collusion_group_size, replace=False)
    
    for voter in colluding_voters:
        voter.reset_voter()
        for project_id in target_project_ids:
            target_project = next(p for p in simulation.round.projects if p.project_id == project_id)
            amount = np.random.uniform(0, voter.balance_op / len(target_project_ids))
            voter.cast_vote(target_project, amount)
    
    colluded_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    simulation.reset_round()
    return colluded_allocations

def measure_impact(baseline_allocations, colluded_allocations):
    difference = np.abs(np.array(baseline_allocations) - np.array(colluded_allocations)).sum()
    return difference

# Example usage
methods = ['mean', 'median', 'quadratic']
simulation = Simulation()
simulation.initialize_round(30_000_000)
simulation.randomize_voters(150, willingness_to_spend=1, laziness_factor=0.6, expertise_factor=0.7)
simulation.randomize_projects(600, coi_factor=0)

for method in methods:
    target_project_ids = np.random.choice([project.project_id for project in simulation.round.projects], size=3, replace=False)
    collusion_impact_score = evaluate_impact_of_collusion(
        simulation, method, quorum=17, min_amount=1500, collusion_group_size=10, target_project_ids=target_project_ids
    )
    print(f'{method} - Impact of Collusion Score: {collusion_impact_score}')



## Incentive Compatibility: Strategy Proofness

In [7]:


def evaluate_strategyproofness(simulation, method, quorum, min_amount, strategy, num_tests=100):
    results = []
    for _ in range(num_tests):
        baseline_allocations, baseline_utilities = run_baseline_simulation(simulation, method, quorum, min_amount)
        strategic_allocations, strategic_utilities = introduce_strategic_voting(simulation, method, quorum, min_amount, strategy)
        is_strategyproof = measure_strategyproofness(baseline_utilities, strategic_utilities)
        results.append(is_strategyproof)
    
    strategyproof_ratio = np.mean(results)
    return strategyproof_ratio

# Helper functions
def calculate_utility(voter, allocations):
    utility = 0
    for vote in voter.votes:
        project_id = vote.project.project_id
        project_allocation = allocations[project_id] if project_id < len(allocations) else 0
        utility += project_allocation * (vote.amount if vote.amount else 0)
    return utility

def run_baseline_simulation(simulation, method, quorum, min_amount):
    simulation.simulate_voting()
    baseline_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    voter_utilities = {
        voter.voter_id: calculate_utility(voter, baseline_allocations)
        for voter in simulation.round.voters
    }
    simulation.reset_round()
    return baseline_allocations, voter_utilities

def introduce_strategic_voting(simulation, method, quorum, min_amount, strategy):
    strategic_voters = simulation.round.voters
    
    for voter in strategic_voters:
        voter.reset_voter()
        strategy(voter, simulation.round.projects)
    
    strategic_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    voter_utilities = {
        voter.voter_id: calculate_utility(voter, strategic_allocations)
        for voter in strategic_voters
    }
    simulation.reset_round()
    return strategic_allocations, voter_utilities

def measure_strategyproofness(baseline_utilities, strategic_utilities):
    strategyproof = True
    for voter_id in baseline_utilities:
        if strategic_utilities[voter_id] > baseline_utilities[voter_id]:
            strategyproof = False
            break
    return strategyproof

# Example strategic voting strategy
def strategic_voting_strategy(voter, projects):
    # Example strategy: Vote only for the highest rated project
    top_project = max(projects, key=lambda p: p.rating)
    voter.cast_vote(top_project, voter.total_op)

# Example usage
methods = ['mean', 'median', 'quadratic']
simulation = Simulation()
simulation.initialize_round(30_000_000)
simulation.randomize_voters(150, willingness_to_spend=1, laziness_factor=0.6, expertise_factor=0.7)
simulation.randomize_projects(600, coi_factor=0)

for method in methods:
    strategyproof_ratio = evaluate_strategyproofness(
        simulation, method, quorum=17, min_amount=1500, strategy=strategic_voting_strategy
    )
    print(f'{method} - Strategyproof Ratio: {strategyproof_ratio}')


mean - Strategyproof Ratio: 0.0
median - Strategyproof Ratio: 0.0
quadratic - Strategyproof Ratio: 0.0


### Group Strategy Proofness

In [10]:


def evaluate_group_strategyproofness(simulation, method, quorum, min_amount, collusion_group_size, target_project_ids, num_tests=100):
    results = []
    for _ in range(num_tests):
        baseline_allocations, baseline_utilities = run_baseline_simulation(simulation, method, quorum, min_amount)
        colluded_allocations, colluded_utilities = introduce_group_collusion(simulation, method, quorum, min_amount, collusion_group_size, target_project_ids)
        is_group_strategyproof = measure_group_strategyproofness(baseline_utilities, colluded_utilities)
        results.append(is_group_strategyproof)
    
    group_strategyproof_ratio = np.mean(results)
    return group_strategyproof_ratio

# Helper functions
def calculate_utility(voter, allocations):
    utility = 0
    for vote in voter.votes:
        project_id = vote.project.project_id
        project_allocation = allocations[project_id] if project_id < len(allocations) else 0
        utility += project_allocation * (vote.amount if vote.amount else 0)
    return utility

def run_baseline_simulation(simulation, method, quorum, min_amount):
    simulation.simulate_voting()
    baseline_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    voter_utilities = {
        voter.voter_id: calculate_utility(voter, baseline_allocations)
        for voter in simulation.round.voters
    }
    simulation.reset_round()
    return baseline_allocations, voter_utilities

def introduce_group_collusion(simulation, method, quorum, min_amount, collusion_group_size, target_project_ids):
    colluding_voters = np.random.choice(simulation.round.voters, size=collusion_group_size, replace=False)
    
    for voter in colluding_voters:
        voter.reset_voter()
        for project_id in target_project_ids:
            target_project = next(p for p in simulation.round.projects if p.project_id == project_id)
            amount = np.random.uniform(0, voter.balance_op / len(target_project_ids))
            voter.cast_vote(target_project, amount)
    
    colluded_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    voter_utilities = {
        voter.voter_id: calculate_utility(voter, colluded_allocations)
        for voter in colluding_voters
    }
    simulation.reset_round()
    return colluded_allocations, voter_utilities

def measure_group_strategyproofness(baseline_utilities, colluded_utilities):
    group_strategyproof = True
    for voter_id in baseline_utilities:
        if voter_id in colluded_utilities and colluded_utilities[voter_id] > baseline_utilities[voter_id]:
            group_strategyproof = False
            break
    return group_strategyproof

# Example usage
methods = ['mean', 'median', 'quadratic']
simulation = Simulation()
simulation.initialize_round(30_000_000)
simulation.randomize_voters(150, willingness_to_spend=1, laziness_factor=0.6, expertise_factor=0.7)
simulation.randomize_projects(600, coi_factor=0)

for method in methods:
    target_project_ids = np.random.choice([project.project_id for project in simulation.round.projects], size=3, replace=False)
    group_strategyproof_ratio = evaluate_group_strategyproofness(
        simulation, method, quorum=17, min_amount=1500, collusion_group_size=10, target_project_ids=target_project_ids
    )
    print(f'{method} - Group Strategyproof Ratio: {group_strategyproof_ratio}')


mean - Group Strategyproof Ratio: 1.0
median - Group Strategyproof Ratio: 1.0
quadratic - Group Strategyproof Ratio: 1.0


### Robustness

In [12]:

def evaluate_robustness(simulation, method, quorum, min_amount, num_changes=1, num_tests=100):
    results = []
    for _ in range(num_tests):
        baseline_allocations = run_baseline_simulation(simulation, method, quorum, min_amount)
        changed_allocations = introduce_random_changes(simulation, method, quorum, min_amount, num_changes)
        distance = measure_distance(baseline_allocations, changed_allocations)
        results.append(distance)
    
    avg_distance = np.mean(results)
    return avg_distance

# Helper functions
def run_baseline_simulation(simulation, method, quorum, min_amount):
    simulation.simulate_voting()
    baseline_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    simulation.reset_round()
    return baseline_allocations

def introduce_random_changes(simulation, method, quorum, min_amount, num_changes=1):
    changed_voter = np.random.choice(simulation.round.voters)
    
    for _ in range(num_changes):
        project = np.random.choice(simulation.round.projects)
        amount = np.random.uniform(0, changed_voter.balance_op)
        changed_voter.cast_vote(project, amount)
    
    changed_allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    simulation.reset_round()
    return changed_allocations

def measure_distance(baseline_allocations, changed_allocations):
    distance = np.linalg.norm(np.array(baseline_allocations) - np.array(changed_allocations))
    return distance

# Example usage
methods = ['mean', 'median', 'quadratic']
simulation = Simulation()
simulation.initialize_round(30_000_000)
simulation.randomize_voters(150, willingness_to_spend=1, laziness_factor=0.6, expertise_factor=0.7)
simulation.randomize_projects(600, coi_factor=0)

for method in methods:
    avg_distance = evaluate_robustness(
        simulation, method, quorum=17, min_amount=1500, num_changes=1
    )
    print(f'{method} - Robustness (Average Distance): {avg_distance}')


mean - Robustness (Average Distance): 2232714.2490736637
median - Robustness (Average Distance): 2399706.473329072
quadratic - Robustness (Average Distance): 2172412.445962213


## Pareto efficiency

In [2]:
def run_simulation(simulation, method, quorum, min_amount):
    simulation.simulate_voting()
    allocations = simulation.round.calculate_allocations(method, quorum, min_amount)
    simulation.reset_round()
    return allocations

def check_pareto_efficiency(voters, allocations, projects):
    pareto_violations = 0
    for i in range(len(projects)):
        for j in range(i + 1, len(projects)):
            project_i = projects[i]
            project_j = projects[j]
            majority_prefers_i = sum(1 for voter in voters if voter.prefers(project_i, project_j)) > len(voters) / 2
            majority_prefers_j = sum(1 for voter in voters if voter.prefers(project_j, project_i)) > len(voters) / 2
            
            if majority_prefers_i and allocations[project_i.project_id] < allocations[project_j.project_id]:
                pareto_violations += 1
            if majority_prefers_j and allocations[project_j.project_id] < allocations[project_i.project_id]:
                pareto_violations += 1
    return pareto_violations
def evaluate_pareto_efficiency(simulation, method, quorum, min_amount, num_tests=100):
    results = []
    for _ in range(num_tests):
        allocations = run_simulation(simulation, method, quorum, min_amount)
        pareto_violations = check_pareto_efficiency(simulation.round.voters, allocations, simulation.round.projects)
        results.append(pareto_violations)
    
    avg_pareto_violations = np.mean(results)
    return avg_pareto_violations

# Example usage
methods = ['mean', 'median', 'quadratic']
simulation = Simulation()
simulation.initialize_round(30_000_000)
simulation.randomize_voters(150, willingness_to_spend=1, laziness_factor=0.6, expertise_factor=0.7)
simulation.randomize_projects(600, coi_factor=0)

for method in methods:
    avg_pareto_violations = evaluate_pareto_efficiency(
        simulation, method, quorum=17, min_amount=1500
    )
    print(f'{method} - Average Pareto Violations: {avg_pareto_violations}')
